In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

# Visualize Prediction Result

In [ ]:
#get data from BQ

In [ ]:
dfResult=pd.concat([dfFeature,dfPrediction],ignore_index=False)
print(dfResult.info())
dfResult

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
plt.title(f"{AssetName} - {input_sequence_length} days ago to predict next {output_sequence_length} days", fontsize=12)
#sns.lineplot(data = dfResult[prediction_col], linewidth=4.0,  markers='o',ax=ax,color='black')
sns.lineplot(data = dfResult.query("Type==@colInput")[prediction_col], linewidth=2.0, ax=ax,color='blue')
sns.lineplot(data = dfResult.query("Type==@colOutput")[prediction_col], linewidth=2.0, ax=ax,color='green')